<a href="https://colab.research.google.com/github/fralfaro/MAT281_2024/blob/main/docs/labs/lab_032.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MAT281 - Laboratorio N°032


Esta semana revisaremos datos del **Índice de Libertad de Prensa** que confecciona cada año la asociación de Reporteros Sin Fronteras.


## Diccionario de datos


|Variable       |Clase               |Descripción |
|:--------------|:-------------------|:-----------|
| codigo_iso | caracter | Código ISO del país |
| pais | caracter | País |
| anio | entero | Año del resultado |
| indice | entero | Puntaje Índice Libertad de Prensa (menor puntaje = mayor libertad de prensa) |
| ranking | entero | Ranking Libertad de Prensa |


## Fuente original y adaptación
Los datos fueron extraídos de [The World Bank](https://tcdata360.worldbank.org/indicators/h3f86901f?country=BRA&indicator=32416&viz=line_chart&years=2001,2019). La fuente original es [Reporteros sin Fronteras](https://www.rsf-es.org/).

Por otro lado, estos archivos han sido modificado intencionalmente para ocupar todo lo aprendido en clases. A continuación, una breve descripción de cada uno de los data frames:

* **libertad_prensa_codigo.csv**: contiene la información codigo_iso/pais. Existe un código que tiene dos valores.
* **libertad_prensa_01.csv**: contiene la información pais/anio/indice/ranking. Los nombres de las columnas estan en mayúscula (antes del año 2010).
* **libertad_prensa_02.csv**: contiene la información pais/anio/indice/ranking. Los nombres de las columnas estan en mayúscula (después del año 2010).




In [1]:
import numpy as np
import pandas as pd

# lectura de datos
archivos_anio = [
    'https://raw.githubusercontent.com/fralfaro/MAT281_2024/main/docs/labs/data/libertad_prensa_01.csv',
    'https://raw.githubusercontent.com/fralfaro/MAT281_2024/main/docs/labs/data/libertad_prensa_02.csv'
 ]
df_codigos = pd.read_csv('https://raw.githubusercontent.com/fralfaro/MAT281_2024/main/docs/labs/data/libertad_prensa_codigo.csv')

 El objetivo es tratar de obtener la mayor información posible de este conjunto de datos. Para cumplir este objetivo debe resolver las siguientes problemáticas:

1. Lo primero será juntar toda la información en un _solo archivo_, para ello necesitamos seguir los siguientes pasos:

 * a) Crear el archivo **df_anio**, que contenga la información de **libertad_prensa_anio.csv** para cada año. Luego, normalice el nombre de las columnas a minúscula.
 * b) Encuentre y elimine el dato que esta duplicado en el archivo **df_codigo**.
 * c) Crear el archivo **df** que junte la información del archivo **df_anio** con **df_codigo** por la columna _codigo_iso_.

> **Hint**: Para juntar por _anio_ ocupe la función **pd.concat**. Para juntar información por columna ocupe **pd.merge**.

In [2]:
#a)

#Archigo df_anio
lista_df = []
for archivo in archivos_anio:
    df_anio = pd.read_csv(archivo)
    df_anio.columns = df_anio.columns.str.lower()
    lista_df.append(df_anio)

df_anio = pd.concat(lista_df, ignore_index=True)

#b)
df_codigos = df_codigos.drop_duplicates(subset='codigo_iso')

#c)
df = pd.merge(df_anio, df_codigos, on='codigo_iso')

df.head()

,codigo_iso,anio,indice,ranking,pais
0,AFG,2001,35.50,59.0,Afghanistán
1,AFG,2002,40.17,78.0,Afghanistán
2,AFG,2003,28.25,49.0,Afghanistán
3,AFG,2004,39.17,62.0,Afghanistán
4,AFG,2005,44.25,67.0,Afghanistán


2. Encontrar:
   * ¿Cuál es el número de observaciones en el conjunto de datos?   
   * ¿Cuál es el número de columnas en el conjunto de datos?   
   * Imprime el nombre de todas las columnas  
   * ¿Cuál es el tipo de datos de cada columna?
   * Describir el conjunto de datos (**hint**: .describe())
    

In [3]:
#Numero de Observaciones
num_observaciones = df.shape[0]
print("Número de observaciones:", num_observaciones)

#Numero de Columnas
num_columnas = df.shape[1]
print("Número de columnas:", num_columnas)

#Nombre de las Columnas
nombres_columnas = df.columns
print("Nombres de las columnas:")
for nombre in nombres_columnas:
    print(nombre)

#Tipo de Dato
tipo_datos = df.dtypes
print("\nTipo de datos de cada columna:")
print(tipo_datos)

#Descripcion del Conjunto de Datos
descripcion = df.describe(include='all')
print("\nDescripción del conjunto de datos:")
print(descripcion)

Número de observaciones: 3060
Número de columnas: 5
Nombres de las columnas:
codigo_iso
anio
indice
ranking
pais

Tipo de datos de cada columna:
codigo_iso     object
anio            int64
indice        float64
ranking       float64
pais           object
dtype: object

Descripción del conjunto de datos:
       codigo_iso         anio        indice        ranking     pais
count        3060  3060.000000   2664.000000    2837.000000     3060
unique        180          NaN           NaN            NaN      179
top           AFG          NaN           NaN            NaN  Nigeria
freq           17          NaN           NaN            NaN       34
mean          NaN  2009.941176    205.782316     477.930913      NaN
std           NaN     5.786024   2695.525264    6474.935347      NaN
min           NaN  2001.000000      0.000000       1.000000      NaN
25%           NaN  2005.000000     15.295000      34.000000      NaN
50%           NaN  2009.000000     28.000000      70.000000      NaN
75%  

3. Desarrolle una función `resumen_df(df)` para encontrar el total de elementos distintos y vacíos por columnas.

In [4]:
# respuesta
def resumen_df(df):
    """
    Función para generar un resumen de un DataFrame que incluye
    el número de elementos distintos y vacíos por columna.

    Args:
    df (pd.DataFrame): DataFrame a resumir.

    Returns:
    pd.DataFrame: DataFrame resumen con el nombre de las columnas,
                  cantidad de elementos distintos y cantidad de elementos vacíos.
    """
    # Crear un DataFrame de resultado con los nombres de las columnas
    nombres = df.columns
    result = pd.DataFrame({'nombres': nombres})

    # Calcular el número de elementos distintos por columna
    result['elementos_distintos'] = 0
    for col in df.columns:
        result.loc[result['nombres'] == col, 'elementos_distintos'] = df[col].nunique()

    # Calcular el número de elementos vacíos (NaN) por columna
    result['elementos_vacios'] = 0
    for col in df.columns:
        result.loc[result['nombres'] == col, 'elementos_vacios'] = df[col].isna().sum()

    return result

In [5]:
# retornar
resumen_df(df)

,nombres,elementos_distintos,elementos_vacios
0,codigo_iso,180,0
1,anio,17,0
2,indice,1550,396
3,ranking,193,223
4,pais,179,0


4. Para los paises latinoamericano, encuentre por año  el país con mayor y menor `indice`.

 * a) Mediante un ciclo _for_.
 * b) Mediante un  _groupby_.

In [14]:
# Lista de países en América
america = [
    'ARG', 'ATG', 'BLZ', 'BOL', 'BRA', 'CAN', 'CHL', 'COL', 'CRI',
    'CUB', 'DOM', 'ECU', 'GRD', 'GTM', 'GUY', 'HND', 'HTI', 'JAM',
    'MEX', 'NIC', 'PAN', 'PER', 'PRY', 'SLV', 'SUR', 'TTO', 'URY',
    'USA', 'VEN'
]

# Filtrar el DataFrame para países de América
df_america = df[df.codigo_iso.isin(america)]

#a)

# Función para obtener máximos o mínimos por año
def pais_indice(df, maximo=True):
    resultado = {}                         #Diccionario para guardar los resultados
    for anio, grupo in df.groupby('anio'):
        if grupo['indice'].notna().any():  # Verifica si hay valores no nulos
            if maximo:
                indice_valor = grupo['indice'].max()
                df_temp = grupo[grupo['indice'] == indice_valor]
            else:
                indice_valor = grupo['indice'].min()
                df_temp = grupo[grupo['indice'] == indice_valor]

            pais = df_temp['pais'].iloc[0] if not df_temp.empty else 'No hay datos'
            indice_valor = indice_valor if not np.isnan(indice_valor) else 0
        else:
            # Caso cuando todos los valores son NaN
            pais = 'No hay datos'
            indice_valor = 0

        # Guardar en el diccionario
        resultado[anio] = [pais, indice_valor]

    return resultado

# Obtener máximos y mínimos por año
max_anio = pais_indice(df_america, maximo=True)
min_anio = pais_indice(df_america, maximo=False)

# Crear un DataFrame para mostrar los resultados
resultados_df = pd.DataFrame({
    'Máximo': [f"{max_anio[anio][0]} ({max_anio[anio][1]})" for anio in max_anio],
    'Mínimo': [f"{min_anio[anio][0]} ({min_anio[anio][1]})" for anio in min_anio]
}, index=max_anio.keys())

# Mostrar la tabla de resultados
resultados_df


,Máximo,Mínimo
2001,Cuba (90.3),Canadá (0.8)
2002,Cuba (97.83),Trinidad y Tobago (1.0)
2003,Argentina (35826.0),Trinidad y Tobago (2.0)
2004,Cuba (87.0),Trinidad y Tobago (2.0)
2005,Cuba (95.0),Bolivia (4.5)
2006,Cuba (96.17),Canadá (4.88)
2007,Cuba (88.33),Canadá (3.33)
2008,Cuba (94.0),Canadá (3.7)
2009,Cuba (78.0),Estados Unidos (6.75)
2012,Cuba (71.64),Jamaica (9.88)


In [23]:
#b)

# Máximos por año
max_index = df_america.groupby('anio').indice.max()  # Se guardan los índices máximos por año
df_max = df_america.merge(max_index, on='anio', suffixes=('', '_max'))  # Agregamos una nueva columna con los índices máximos por año
df_max = df_max[df_max.indice == df_max.indice_max].drop(['ranking', 'indice_max'], axis=1, errors='ignore')  # Filtramos las filas con el máximo índice

# Mínimos por año
min_index = df_america.groupby('anio').indice.min()  # Se guardan los índices mínimos por año
df_min = df_america.merge(min_index, on='anio', suffixes=('', '_min'))  # Agregamos una nueva columna con los índices mínimos por año
df_min = df_min[df_min.indice == df_min.indice_min].drop(['ranking', 'indice_min'], axis=1, errors='ignore')  # Filtramos las filas con el mínimo índice

# Combinar los resultados en un solo DataFrame
df_resultados2 = pd.merge(
    df_max[['anio', 'pais', 'indice']].rename(columns={'pais': 'Máximo', 'indice': 'Índice Máx'}),
    df_min[['anio', 'pais', 'indice']].rename(columns={'pais': 'Mínimo', 'indice': 'Índice Mín'}),
    on='anio',
    how='outer'
).sort_values(by='anio')

# Mostrar la tabla de resultados
df_resultados2


,anio,Máximo,Índice Máx,Mínimo,Índice Mín
0,2001,Cuba,90.30,Canadá,0.80
1,2002,Cuba,97.83,Trinidad y Tobago,1.00
2,2003,Argentina,35826.00,Trinidad y Tobago,2.00
3,2004,Cuba,87.00,Trinidad y Tobago,2.00
4,2005,Cuba,95.00,Bolivia,4.50
5,2005,Cuba,95.00,Canadá,4.50
6,2006,Cuba,96.17,Canadá,4.88
7,2007,Cuba,88.33,Canadá,3.33
8,2008,Cuba,94.00,Canadá,3.70
9,2009,Cuba,78.00,Estados Unidos,6.75


5. Para cada _país_, muestre el _indice_ máximo que alcanzo por _anio_. Para los datos nulos, rellene con el valor **0**.

> **Hint**: Utilice la función **pd.pivot_table**.



In [24]:
#Indice Maximo
df_indice_max = pd.pivot_table(df_america, values='indice', index=['pais'], columns=['anio'], aggfunc='max')
df_indice_max.fillna(0, inplace=True)
df_indice_max

anio,2001,2002,2003,2004,2005,2006,2007,2008,2009,2012,2013,2014,2015,2017,2018,2019
pais,,,,,,,,,,,,,,,,
Antigua y Barbuda,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,20.81,0.00,0.00,0.00,0.00,0.00
Argentina,12.0,15.17,35826.00,13.67,17.30,24.83,14.08,11.33,16.35,25.67,25.27,26.11,25.09,25.07,26.05,28.30
Belize,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,17.05,18.54,20.61,23.43,24.55,27.50
Bolivia,14.5,9.67,20.00,9.67,4.50,21.50,28.20,24.17,28.13,32.80,31.04,31.29,31.78,33.88,32.45,35.38
Brasil,18.8,16.75,16.50,14.50,17.17,25.25,18.00,15.88,16.60,32.75,34.03,31.93,32.62,33.58,31.20,32.79
Canadá,0.8,1.83,3.33,4.50,4.50,4.88,3.33,3.70,7.00,12.69,10.99,10.99,15.26,16.53,15.28,15.69
Chile,6.5,6.83,10.00,11.75,11.63,12.13,11.50,10.50,10.50,26.24,25.80,23.00,19.23,20.53,22.69,25.65
Colombia,40.8,49.17,47.38,40.17,44.75,42.33,35.50,40.13,51.50,37.48,36.68,39.08,44.11,41.47,41.03,42.82
Costa Rica,4.3,3.83,7.63,8.50,6.67,6.50,5.10,8.00,8.08,12.08,12.23,12.26,11.10,11.93,14.01,12.24
